`KDD2024 Tutorial / A Hands-On Introduction to Time Series Classification and Regression`

# Part9: Hybrid Estimators for Time Series Machine Learning in `aeon`

`aeon` contains two hard coded hybrid classifiers, HIVE-COTEV1 (HC1) and HIVE-COTEV2
(HC2). We will just use HC2 in this example.


In [1]:
# There are some deprecation warnings present in the notebook, we will ignore them.
# Remove this cell if you are interested in finding out what is changing soon, for
# aeon there will be big changes in out v1.0.0 release!
import warnings

warnings.filterwarnings("ignore")
import numpy as np
from aeon.datasets import load_classification
trainX = np.random.random((20,3, 100)) # Random collection of 20 series, each with 3 channels and 100 timepoints
train_labels = np.random.randint(0, 2, 20) # Random labels
train_target = np.random.random(20) # Random regression target
# load EEG example here

from aeon.classification.distance_based import KNeighborsTimeSeriesClassifier
from aeon.datasets import load_from_tsfile

# Load the univariate EEG TSC dataset as a 3D numpy array
X_train_utsc, y_train_utsc = load_from_tsfile("./data/KDD_UTSC_TRAIN.ts")
X_test_utsc, y_test_utsc = load_from_tsfile("./data/KDD_UTSC_TEST.ts")

print("Single channel EEG TSC problem train shape:", X_train_utsc.shape)
print("Single channel EEG TSC problem test shape:", X_test_utsc.shape)

# Load the univariate EEG TSC dataset as a 3D numpy array
X_train_mtsc, y_train_mtsc = load_from_tsfile("./data/KDD_MTSC_TRAIN.ts")
X_test_mtsc, y_test_mtsc = load_from_tsfile("./data/KDD_MTSC_TEST.ts")

print("Four channel EEG TSC problem train shape:", X_train_mtsc.shape)
print("Single channel EEG TSC problem test shape:", X_test_mtsc.shape)

# Load the univariate EEG TSER dataset as a 3D numpy array
X_train_utser, y_train_utser = load_from_tsfile("./data/KDD_UTSER_TRAIN.ts")
X_test_utser, y_test_utser = load_from_tsfile("./data/KDD_UTSER_TEST.ts")

print("Single channel EEG TSER problem train shape:", X_train_utsc.shape)
print("Single channel EEG TSER problem test shape:", X_test_utsc.shape)

# Load the univariate EEG TSER dataset as a 3D numpy array
X_train_mtser, y_train_mtser = load_from_tsfile("./data/KDD_MTSER_TRAIN.ts")
X_test_mtser, y_test_mtser = load_from_tsfile("./data/KDD_MTSER_TEST.ts")

print("Four channel EEG TSER problem train shape:", X_train_mtser.shape)
print("Single channel EEG TSER problem test shape:", X_test_mtser.shape)


Single channel EEG TSC problem train shape: (40, 1, 100)
Single channel EEG TSC problem test shape: (40, 1, 100)
Four channel EEG TSC problem train shape: (40, 4, 100)
Single channel EEG TSC problem test shape: (40, 4, 100)
Single channel EEG TSER problem train shape: (40, 1, 100)
Single channel EEG TSER problem test shape: (40, 1, 100)
Four channel EEG TSER problem train shape: (72, 4, 100)
Single channel EEG TSER problem test shape: (72, 4, 100)


HC2 is not designed to be fast on small examples. For ease of example we can set the contract
to reduce the run time and set number of jobs and can be recovered like this.

In [2]:
from aeon.classification.hybrid import HIVECOTEV2
hc2 = HIVECOTEV2(n_jobs  = -1, time_limit_in_minutes=2)
hc2.fit(X_train_utsc, y_train_utsc)
print("Single channel EEG TSC problem accuracy:", hc2.score(X_test_utsc, y_test_utsc))
print("STC weight = " + str(hc2.stc_weight_))
print("DrCIF weight = " + str(hc2.drcif_weight_))
print("Arsenal weight = " + str(hc2.arsenal_weight_))
print("TDE weight = " + str(hc2.tde_weight_))

Single channel EEG TSC problem accuracy: 0.55
STC weight = 0.586181640625
DrCIF weight = 0.8145062499999999
Arsenal weight = 0.7320941406250001
TDE weight = 0.4632503906249999


## Make your own hybrid estimator

HC2 is hard coded as an estimator so we can recreate results and use it as the
authors proposed it. However, you can make your own weighted ensemble.

In [3]:
from aeon.classification import DummyClassifier
from aeon.classification.convolution_based import RocketClassifier
from aeon.classification.feature_based import FreshPRINCEClassifier
from aeon.classification.compose._ensemble import ClassifierEnsemble

clf = [DummyClassifier(), RocketClassifier(), FreshPRINCEClassifier()]
ens = ClassifierEnsemble(clf, weights=[0.1, 0.3, 0.6])
ens.fit(X_train_utsc, y_train_utsc)
print("Some ensemble accuracy:", ens.score(X_test_utsc, y_test_utsc))

Some ensemble accuracy: 0.85
